In [1]:
%pip uninstall -y lightgbm pyarrow cffi pycparser autogluon autogluon.tabular
%pip install -U pip setuptools wheel
%pip install -U --no-cache-dir cffi pycparser
%pip install -U --no-cache-dir pyarrow lightgbm
%pip install -U --no-cache-dir autogluon.tabular

Found existing installation: cffi 2.0.0
Uninstalling cffi-2.0.0:
  Successfully uninstalled cffi-2.0.0
Found existing installation: pycparser 3.0
Uninstalling pycparser-3.0:
  Successfully uninstalled pycparser-3.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [cffi]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 MB 204.1 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 192.9 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [lightgbm]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 175.1 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 MB 191.0 MB/s  0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 23.0.0
    Uninstalling pyarrow-23.0.0:
      Successfully uninstalled pyarrow-23.0.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9/9 [autogluon.tabular]


In [2]:
!pip install autogluon.tabular[realmlp]==1.5.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.5/849.5 kB 15.0 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 44.5 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [pytabkit]


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd

In [5]:
INPUT_PATH = '/data/processed'
OUTPUT_PATH = '/runs/'

df_train = pd.read_csv('/content/drive/MyDrive/ICML-transactions/data_retail/retail-df_train.csv')
df_val = pd.read_csv('/content/drive/MyDrive/ICML-transactions/data_retail/retail-df_val.csv')
df_test = pd.read_csv('/content/drive/MyDrive/ICML-transactions/data_retail/retail-df_test.csv')

In [6]:
df_train.drop('Unnamed: 0', axis=1, inplace=True)
df_val.drop('Unnamed: 0', axis=1, inplace=True)
df_test.drop('Unnamed: 0', axis=1, inplace=True)

In [7]:
# fix types

for t in ['CustomerID']:
    df_train[t] = df_train[t].astype('category')
    df_val[t] = df_val[t].astype('category')
    df_test[t] = df_test[t].astype('category')

In [8]:
tasks = ['unique_StockCode',
       'count_gt_75_InvoiceNo', 'count_eq_15_InvoiceNo',
       'double_InvoiceNo_StockCode_23084', 'duplicate_CustomerID_StockCode']

In [9]:
import pandas as pd

def scores_dict_to_df(scores_dict):
    """
    Convert a nested scores dictionary into a pandas DataFrame.

    Parameters
    ----------
    scores_dict : dict
        Dictionary of the form:
        {
            "test_i": [ {metric: value, ...}, {metric: value, ...} ],
            ...
        }

    Returns
    -------
    pd.DataFrame
        DataFrame with one row per (task, run) and one column per metric.
    """
    rows = []

    for task, results in scores_dict.items():
        for run_idx, metrics in enumerate(results):
            row = {
                "task": task,
                "run": run_idx,
            }
            # Ensure numpy scalars are converted to Python scalars
            row.update({k: float(v) for k, v in metrics.items()})
            rows.append(row)

    return pd.DataFrame(rows)


# training

In [10]:
df_train_task = df_train.drop(tasks, axis=1)
df_val_task = df_val.drop(tasks, axis=1)
df_test_task = df_test.drop(tasks, axis=1)

## Uniqueness

In [11]:
from autogluon.tabular import TabularPredictor
from autogluon.features.generators import IdentityFeatureGenerator


"""'unique_StockCode',
       'count_gt_75_InvoiceNo', 'count_eq_15_InvoiceNo',
       'double_InvoiceNo_StockCode_23084', 'duplicate_CustomerID_StockCode'"""


t = 'unique_StockCode'
df_train_task[t] = df_train[t]
df_val_task[t] = df_val[t]
df_test_task[t] = df_test[t]

predictor_ag = TabularPredictor(
    label=t,
    eval_metric="f1"
).fit(
    train_data=df_train_task,
    tuning_data=df_val_task,
    time_limit=300,
    hyperparameters={
        "GBM": {},       # LightGBM
        "REALMLP": {},   # RealMLP
    },
    presets=None,
)

lb_ag = predictor_ag.leaderboard(df_test_task, silent=True, extra_metrics=["precision", "roc_auc"])


scores_dict ={}

scores_lgbm = predictor_ag.evaluate(df_test_task, "LightGBM")
scores_mlp = predictor_ag.evaluate(df_test_task, "RealMLP")

scores_dict[f"test"] = [scores_lgbm, scores_mlp]

results = scores_dict_to_df(scores_dict)

# replace value 0 and 1 in results for lightgbm and realmlp
results["run"] = results["run"].replace({
    0: "LightGBM",
    1: "RealMLP"
})

results.to_csv(OUTPUT_PATH+f"Retail-Tabular_{t}.csv")

df_train_task.drop(t, axis=1, inplace=True)
df_val_task.drop(t, axis=1, inplace=True)
df_test_task.drop(t, axis=1, inplace=True)

No path specified. Models will be saved in: "AutogluonModels/ag-20260128_234430"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.5.0
Python Version:     3.12.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Thu Oct  2 10:42:05 UTC 2025
CPU Count:          2
Pytorch Version:    2.9.0+cpu
CUDA Version:       CUDA is not available
Memory Avail:       11.16 GB / 12.67 GB (88.1%)
Disk Space Avail:   204.41 GB / 225.83 GB (90.5%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets. Defaulting to `'medium'`...
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='extreme'  : New in v1.5: The state-of-the-art for tabular data. Massively better than 'best' on datasets <100000 samples by using new Tabular Foundation Models (TFMs) meta-learned on https://tabarena.a

KeyError: 'unique_StockCode'

# Predict with TabPFN

In [ ]:
## Base library Installation
# Install Baselines for model comparison
!uv pip install catboost xgboost

# Install the datasets library for loading example data
!uv pip install datasets

# Install rich for better and more readable printing
!uv pip install rich

## TabPFN Installation optimized for Google Colab
# Install the TabPFN Client library
!uv pip install tabpfn-client

# Install TabPFN extensions for additional functionalities
!uv pip install 'tabpfn-extensions[all]'

# Install tabpfn
!uv pip install tabpfn

Using Python 3.12.12 environment at: /usr
Resolved 21 packages in 501ms
Prepared 1 package in 2.35s
Installed 1 package in 15ms
 + catboost==1.2.8
Using Python 3.12.12 environment at: /usr
Audited 1 package in 140ms
Using Python 3.12.12 environment at: /usr
Audited 1 package in 96ms
Using Python 3.12.12 environment at: /usr
Resolved 33 packages in 278ms
Prepared 4 packages in 105ms
Installed 4 packages in 4ms
 + backoff==2.2.1
 + password-strength==0.0.3.post2
 + sseclient-py==1.8.0
 + tabpfn-client==0.2.8
Using Python 3.12.12 environment at: /usr
Resolved 80 packages in 1.29s
Prepared 15 packages in 634ms
Uninstalled 5 packages in 55ms
Installed 15 packages in 38ms
 - autogluon-common==1.5.0
 + autogluon-common==1.4.0
 - autogluon-core==1.5.0
 + autogluon-core==1.4.0
 - autogluon-features==1.5.0
 + autogluon-features==1.4.0
 - autogluon-tabular==1.5.0
 + autogluon-tabular==1.4.0
 + ecos==2.0.14
 + eval-type-backport==0.3.1
 + galois==0.4.10
 + posthog==6.9.3
 - requests==2.32.4
 + req

In [ ]:
from tabpfn_client import TabPFNClassifier

# Now you can use it like any other sklearn classifier
# model = TabPFNClassifier()
print("TabPFNClassifier imported successfully.")

TabPFNClassifier imported successfully.


In [ ]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, log_loss, roc_auc_score, f1_score

In [ ]:
X_train = df_val.drop(tasks, axis=1)
X_test = df_test.drop(tasks, axis=1)

In [ ]:
t = 'duplicate_CustomerID_StockCode' # ['unique_StockCode','count_gt_75_InvoiceNo', 'count_eq_15_InvoiceNo',double_InvoiceNo_StockCode_23084', 'duplicate_CustomerID_StockCode']

y_train = df_val[t]
y_test = df_test[t]

tabpfn_classifier = TabPFNClassifier(random_state=42)

In [ ]:
tabpfn_classifier.fit(X_train, y_train)

TabPFNClassifier(random_state=42)

In [ ]:
# .predict() and .predict_proba() will use the tuned settings
preds = tabpfn_classifier.predict(X_test)
preds_proba = tabpfn_classifier.predict_proba(X_test)

f1_tabpfn = f1_score(y_test, preds)
roc_auc_tabpfn = roc_auc_score(y_test, preds_proba[:,1])

print("F1 score", f1_tabpfn)
print("roc_auc", roc_auc_tabpfn)

Processing: 100%|██████████| [00:02<00:00]
Processing: 100%|██████████| [00:03<00:00]


In [ ]:
results_tabpfn = pd.DataFrame({
"model": ["TabPFN2.5"],
"f1_score": [f1_tabpfn],
"roc_auc": [roc_auc_tabpfn]
})

results_tabpfn.to_csv(OUTPUT_PATH+f"Retail-TABPFN_{t}.csv")

/tmp/ipython-input-3542613203.py:5: RuntimeWarning: invalid value encountered in divide
  f1 = 2 * precision[1:] * recall[1:] / (precision[1:] + recall[1:])


0.007722007722007722